In [1]:
import pathlib
import fire as fire
from B_VAE.VAE_v2 import b_encodeco
from Train_utils import train_utils
from B_VAE.Utils_imp_VAE import loss_fn
from Train_utils.train_utils import train,test,K_fold_train

import sys
import os
sys.path.append(os.path.join("..","Data_prep"))
from torchvision import transforms
from Custom_dataloader import *
from Transforms import phantom_segmentation
from Transforms import multi_image_resize
from Transforms import multi_ToTensor
from Transforms import output_transform
from Transforms import rgb_normalize

import torch

In [3]:
DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_1,volume=Paltas_DataBase/Data_Base_v2"

d_tt=transforms.Compose([
        phantom_segmentation(False),
        rgb_normalize(ImType=['PhantomRGB']),
        multi_image_resize(ImType=['PhantomRGB'],size=(100,100)),
        multi_ToTensor(ImType=['PhantomRGB']),
        #output_transform()
        ])

In [4]:
datab=Dataset_direct(root_dir=DB,ImType=['PhantomRGB'],Intersec=False,transform=d_tt)

In [5]:
device='cuda'
T_ID="VAE_v2_3"
pth=os.path.join(str(pathlib.Path().absolute()),"results",T_ID)

In [6]:
model=b_encodeco(image_dim=int(100),
                 image_channels=3,
                 repr_sizes=[5,8,10],
                 layer_sizes=[100],
                 latent_space_size=20,
                 conv_kernel_size=15,
                 conv_pooling=False,
                 conv_batch_norm=True,
                 NN_batch_norm=True,
                 stride=2,
                device=device)

In [7]:
model.to(device)

b_encodeco(
  (encoder_conv): b_encoder_conv(
    (im_layers): ModuleList(
      (0): set_conv(
        (comp_layer): ModuleList(
          (0): Conv2d(3, 5, kernel_size=(15, 15), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): set_conv(
        (comp_layer): ModuleList(
          (0): Conv2d(5, 8, kernel_size=(15, 15), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): set_conv(
        (comp_layer): ModuleList(
          (0): Conv2d(8, 10, kernel_size=(15, 15), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (encoder_NN_mu): NeuralNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): NN_layer(
          (layer): ModuleList(
            (0): L

In [8]:
dataloader_train=torch.utils.data.DataLoader(datab,batch_size=5,shuffle=False,num_workers=5)

In [9]:
for idx,batch in enumerate(dataloader_train):
    print(idx)
    if idx==1:
        break

In [11]:
batch['PhantomRGB'].shape

torch.Size([5, 3, 100, 100])

In [12]:
r_img,mu,sig=model(batch["PhantomRGB"].to(device))

In [53]:
from torch import nn
import torch.nn.functional as F
import torch

def loss_fn(r_x,x,mu,sig):
    BCE=F.binary_cross_entropy(r_x,x,reduction='mean')
    KLD=-0.5*torch.mean(1+sig-mu.pow(2)-sig.exp())
    #KLD=-0.5*torch.sum(1+sig-mu.pow(2)-sig.exp())
    return BCE+KLD,BCE,KLD

In [54]:
loss,bce,kld=loss_fn(r_img,batch["PhantomRGB"].to(device),mu,sig)

In [55]:
loss

tensor(1.2354, device='cuda:0', grad_fn=<AddBackward0>)

In [56]:
bce

tensor(0.7744, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [57]:
kld

tensor(0.4609, device='cuda:0', grad_fn=<MulBackward0>)

In [42]:
(1+sig-mu.pow(2)-sig.exp())

tensor([[-6.3392e-01,  0.0000e+00, -2.6897e+00, -5.0880e-01,  0.0000e+00,
         -3.5916e-02,  0.0000e+00, -2.2640e+00, -2.4324e+00,  0.0000e+00,
         -1.0753e-01, -5.4202e-01, -3.4461e-02,  0.0000e+00, -2.5388e-01,
         -1.5793e+00,  0.0000e+00, -2.9524e+00,  0.0000e+00,  0.0000e+00],
        [-1.0759e+00, -2.6869e-01, -7.4587e-01, -4.2679e-01, -6.1753e-01,
         -3.1100e-02, -1.4006e+00, -3.2391e-02,  0.0000e+00, -3.4836e+00,
         -1.0252e-05, -1.7863e-01, -7.7413e-02, -1.6180e+00, -1.1115e+00,
         -8.3242e-01,  0.0000e+00, -7.4880e-01,  0.0000e+00, -1.8505e-03],
        [-1.6952e+00, -3.9729e+00,  0.0000e+00, -3.7499e-02,  0.0000e+00,
         -1.2590e+00, -6.5761e-01, -1.2705e-02,  0.0000e+00,  0.0000e+00,
         -1.4285e+00, -1.5866e+00, -1.5741e+00, -6.9874e-01, -9.0892e-01,
         -2.2884e+00, -6.9883e-01,  0.0000e+00, -2.2634e+00,  0.0000e+00],
        [-8.4756e-01, -3.6447e-02, -2.0547e-01,  0.0000e+00, -3.0932e-01,
         -3.8283e+00, -6.1752e-02, 